In [ ]:
from pykalman import KalmanFilter
import numpy as np
import matplotlib.pyplot as plt
import time
from rdp import rdp
%matplotlib inline
plt.rcParams['figure.figsize'] = [10, 10]

In [ ]:
import gpxpy

with open('m8t.gpx') as fh:
    gpx_file = gpxpy.parse(fh)

In [ ]:
segment = gpx_file.tracks[0].segments[1]

import pandas as pd
coords = pd.DataFrame([{'idx': i,
                        'lat': p.latitude, 
                        'lon': p.longitude, 
                        'ele': p.elevation,
                        'speed': p.speed,
                        'time': p.time} for i, p in enumerate(segment.points)])
coords.set_index('time', inplace=True)
coords.head()

In [ ]:
measurements_all = np.ma.masked_invalid(coords[['lon', 'lat', 'speed']].values)
times_all = range(measurements_all.shape[0])

In [ ]:
plt.plot(measurements_all[:, 0], measurements_all[:, 1], 'bo')

In [ ]:
#coords_part = coords[['lon', 'lat', 'speed']][750:880].values
coords_part = coords[['lon', 'lat', 'speed']][750:980].values
measurements_part = np.ma.masked_invalid(coords_part)
plt.plot(range(measurements_part.shape[0]), measurements_part[:, 2], 'bo')

In [ ]:
simple_coords = rdp(coords_part[:,[0,1]], algo="iter", epsilon=1e-5)
measurements_reduced = np.ma.masked_invalid(simple_coords)
#measurements_reduced = measurements_part[measurements_part[:,2] > 1]
plt.plot(measurements_part[:, 0], measurements_part[:, 1], 'go',
         measurements_reduced[:, 0], measurements_reduced[:, 1], 'rx',
        )

In [ ]:
#measurements = np.ma.masked_invalid(coords[['lon', 'lat', 'speed']][750:880].values)
#times = range(measurements.shape[0])
#measurements = np.ma.masked_invalid(coords[['lon', 'lat', 'speed']][780:810].values)
#times = range(measurements.shape[0])
measurements = measurements_part
times = range(measurements.shape[0])


In [ ]:
plt.plot(measurements[:, 0], measurements[:, 1], 'bo')

In [ ]:
#plt.plot(times, measurements[:, 0], 'bo')

In [ ]:
#plt.plot(times, measurements[:, 1], 'bo')

In [ ]:
#plt.plot(times, measurements[:, 2], 'bo')

In [ ]:
xy = measurements[:, [0,1]]


In [ ]:
initial_state_mean = [xy[0, 0],
                      0,
                      xy[0, 1],
                      0]

transition_matrix = [[1, 1, 0, 0],
                     [0, 1, 0, 0],
                     [0, 0, 1, 1],
                     [0, 0, 0, 1]]

observation_matrix = [[1, 0, 0, 0],
                      [0, 0, 1, 0]]

kf1 = KalmanFilter(transition_matrices = transition_matrix,
                  observation_matrices = observation_matrix,
                  initial_state_mean = initial_state_mean)

kf1 = kf1.em(xy, n_iter=100)
(smoothed_state_means, smoothed_state_covariances) = kf1.filter(xy)

kf2 = KalmanFilter(transition_matrices = transition_matrix,
                  observation_matrices = observation_matrix,
                  initial_state_mean = initial_state_mean,
                  observation_covariance = 10*kf1.observation_covariance,
                  em_vars=['transition_covariance', 'initial_state_covariance'])

kf2 = kf2.em(xy, n_iter=100)
(smoothed_state_means2, smoothed_state_covariances2)  = kf2.filter(xy)


In [ ]:
fig=plt.figure()
plt.plot(xy[:, 0], xy[:, 1], 'bo',
         smoothed_state_means[:, 0], smoothed_state_means[:, 2], 'g--',
         smoothed_state_means2[:, 0], smoothed_state_means2[:, 2], 'r--')
plt.show()


In [ ]:
import mplleaflet
mplleaflet.display(fig=fig)

In [ ]:
#plt.plot(times, xy[:, 0], 'bo',
#         times, smoothed_state_means[:, 0], 'r--')


In [ ]:
#plt.plot(times, xy[:, 1], 'bo',
#         times, smoothed_state_means[:, 2], 'r--')

In [ ]:
gpx = gpxpy.gpx.GPX()

# Create first track in our GPX:
gpx_track = gpxpy.gpx.GPXTrack()
gpx.tracks.append(gpx_track)

# Create first segment in our GPX track:
gpx_segment = gpxpy.gpx.GPXTrackSegment()
gpx_track.segments.append(gpx_segment)

# Create points:
for point in smoothed_state_means:
    gpx_segment.points.append(gpxpy.gpx.GPXTrackPoint(point[2], point[0]))

with open("output.gpx", "w") as f:
    f.write( gpx.to_xml())